In [1]:
import os

import pandas as pd
import numpy as np

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.theta import ThetaForecaster

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', ValueWarning)


os.chdir(os.path.abspath(''))
df_train_total = pd.read_csv("../data/df_train_total.csv")
df_test_total = pd.read_csv("../data/df_test_total.csv")
df_merged = pd.read_csv("../data/df_merged.csv")

In [2]:
def generate_ds_arima(label_df, input_seq_len=48, tau=12):
    col_labels =  'wl_1018680' # ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
    
    tmp_df = np.array(label_df.loc[label_df['month'].isin([2,3]), col_labels])
    
    n = tmp_df.shape[0] - input_seq_len - tau 
    
    conti_input = np.zeros((n, input_seq_len), dtype=np.float32)
    label = np.zeros((n, tau))

    for j in range(n):
        conti_input[j, :] = tmp_df[j:(j+input_seq_len)]
        label[j, :] = tmp_df[(j+input_seq_len):(j+input_seq_len+tau)]

    return conti_input, label

### ARIMA

In [3]:
year = "2016"

df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

fh = ForecastingHorizon(np.arange(1, 13), is_relative=True)

arima_forecaster = ARIMA()

arima_results = []

y_train = df_merged.loc[df_merged["month"].isin([0, 1]), "wl_1018680"]
arima_forecaster.fit(y_train)

y_input, label = generate_ds_arima(df_merged)

for i in range(y_input.shape[0]):
    arima_forecaster.update(y_input[i], update_params=False)
    tmp_result = arima_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
    
    arima_results.append(tmp_result.values[np.newaxis, ...])

arima_results = np.concatenate(arima_results, axis=0)

print(np.maximum(0.9 * (label - arima_results[..., 3]), (1-0.9)*(arima_results[..., 3] -label )).mean()/1000) # 0.0031
print(np.mean(label < arima_results[..., 3]), 0.9 - np.mean(label < arima_results[..., 3]))  ### 0.638

print(np.maximum(0.7 * (label - arima_results[..., 2]), (1-0.7)*(arima_results[..., 2] -label )).mean()/1000) # 0.0051
print(np.mean(label < arima_results[..., 2]), 0.7 - np.mean(label < arima_results[..., 2])) ### 0.640

print(np.maximum(0.5 * (label - arima_results[..., 1]), (1-0.5)*(arima_results[..., 1] -label )).mean()/1000)  # 0.0059
print(np.mean(label < arima_results[..., 1]), 0.5- np.mean(label < arima_results[..., 1])) ### 0.894

0.028615709296531486
0.5043767507002801 0.3956232492997199
0.03079209886110186
0.3443043884220355 0.3556956115779645
0.026915412746828717
0.2340686274509804 0.2659313725490196


### ETS

In [4]:
ets_results = []
ets_forecaster = AutoETS(auto=True, n_jobs=-1)  

ets_forecaster.fit(y_train)

for i in range(y_input.shape[0]):
    ets_forecaster.update(y_input[i], update_params=False)
    tmp_result = ets_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
    
    ets_results.append(tmp_result.values[np.newaxis, ...])
    
ets_results = np.concatenate(ets_results, axis=0)

print(np.maximum(0.9 * (label - ets_results[..., 3]), (1-0.9)*(ets_results[..., 3] -label )).mean()/1000) 
print(np.mean(label < ets_results[..., 3]), 0.9 - np.mean(label < ets_results[..., 3])) 

print(np.maximum(0.7 * (label - ets_results[..., 2]), (1-0.7)*(ets_results[..., 2] -label )).mean()/1000) 
print(np.mean(label < ets_results[..., 2]), 0.7 - np.mean(label < ets_results[..., 2])) 

print(np.maximum(0.5 * (label - ets_results[..., 1]), (1-0.5)*(ets_results[..., 1] -label )).mean()/1000)  
print(np.mean(label < ets_results[..., 1]), 0.5- np.mean(label < ets_results[..., 1]))

/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplemented

0.025455679993808385
0.5646008403361344 0.3353991596638656
0.02603372129010405
0.4884453781512605 0.21155462184873947
0.023145632321373028
0.4279878618113912 0.0720121381886088


/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: AutoETS can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplemented

### Theta

In [5]:
theta_results = []
theta_forecaster = ThetaForecaster()    

theta_forecaster.fit(y_train)

for i in range(y_input.shape[0]):
    theta_forecaster.update(y_input[i], update_params=False)
    tmp_result = theta_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
    
    theta_results.append(tmp_result.values[np.newaxis, ...])

theta_results = np.concatenate(theta_results, axis=0)

print(np.maximum(0.9 * (label - theta_results[..., 3]), (1-0.9)*(theta_results[..., 3] -label )).mean()/1000)
print(np.mean(label < theta_results[..., 3]), 0.9 - np.mean(label < theta_results[..., 3]))  

print(np.maximum(0.7 * (label - theta_results[..., 2]), (1-0.7)*(theta_results[..., 2] -label )).mean()/1000) 
print(np.mean(label < theta_results[..., 2]), 0.7 - np.mean(label < theta_results[..., 2]))

print(np.maximum(0.5 * (label - theta_results[..., 1]), (1-0.5)*(theta_results[..., 1] -label )).mean()/1000) 
print(np.mean(label < theta_results[..., 1]), 0.5- np.mean(label < theta_results[..., 1])) 

/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: Use

0.019275009184780593
0.742530345471522 0.15746965452847805
0.02433705048799746
0.5724789915966386 0.12752100840336134
0.023151228893861765
0.42775443510737626 0.07224556489262374


/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: UserWarning: NotImplementedWarning: ThetaForecaster can not accept new data when update_params=False. Call with update_params=True to refit with new data.
  warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ts/lib/python3.9/site-packages/sktime/forecasting/base/adapters/_statsmodels.py:67: Use

### LightGBM

In [6]:
import lightgbm as lgb 
from sklearn.multioutput import MultiOutputRegressor

def generate_ts_data_train_for_lgb(label_df, term_list, input_seq_len=48, tau=12):
    conti_input_list = []
    label_list = []
    col_labels =  ['wl_1018680'] 

    for i in label_df['year'].unique():
        tmp_df = np.array(label_df.loc[label_df['month'].isin([0, 1]), :])
        tmp_label_df = np.array(label_df.loc[label_df['month'].isin([0, 1]), col_labels])        
        n = tmp_df.shape[0] - input_seq_len - tau 
        covariate = np.zeros((n, input_seq_len, tmp_df.shape[1] - 4 ))   
        label = np.zeros((n, tau, len(col_labels)))
        
        for j in range(n):
            covariate[j, :, :] = tmp_df[j:(j+input_seq_len), 4:]
            label[j, :, :] = tmp_label_df[(j+input_seq_len):(j+input_seq_len+tau), :]/1000

        conti_input_list.append(covariate)
        label_list.append(label)
    
    total_conti_input = np.concatenate(conti_input_list, axis=0)
    total_label = np.concatenate(label_list, axis=0)
    
    return total_conti_input.reshape(-1, total_conti_input.shape[1] * total_conti_input.shape[2]), np.squeeze(total_label)

def generate_ts_data_test_for_lgb(label_df, term_list, input_seq_len=48, tau=12):
    conti_input_list = []
    label_list = []
    col_labels =  ['wl_1018680'] 
    
    for i in label_df['year'].unique():
        tmp_df = np.array(label_df.loc[label_df['month'].isin([2, 3]), :])
        tmp_label_df = np.array(label_df.loc[label_df['month'].isin([2, 3]), col_labels])
        n = tmp_df.shape[0] - input_seq_len - tau 
    
        covariate = np.zeros((n, input_seq_len, tmp_df.shape[1] - 4 ))     
        label = np.zeros((n, tau, len(col_labels)))

        for j in range(n):
            covariate[j, :, :] = tmp_df[j:(j+input_seq_len), 4:]
            label[j, :, :] = tmp_label_df[(j+input_seq_len):(j+input_seq_len+tau), :]/1000

        conti_input_list.append(covariate)
        label_list.append(label)
    
    total_conti_input = np.concatenate(conti_input_list, axis=0)
    total_label = np.concatenate(label_list, axis=0)
    
    return total_conti_input.reshape(-1, total_conti_input.shape[1] * total_conti_input.shape[2]), np.squeeze(total_label) # label

In [7]:
year = "2016"

from sktime.transformations.series.fourier import FourierFeatures

df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

term_list = 4
transformer = FourierFeatures(sp_list=[24], fourier_terms_list=[term_list])

df_merged_lgb = pd.concat([df_merged, transformer.fit_transform(df_merged[["wl_1018680"]])], axis=1)
train_input_lgb, label_lgb = generate_ts_data_train_for_lgb(df_merged_lgb, term_list)
train_input_lgb.shape
label_lgb.shape

alphas = [0.5, 0.7, 0.9]

model_list = []

for alpha in alphas:
    params = {
        "objective": "quantile",
        "alpha": alpha,
        "boosting": "gbdt",

    }
    gbm = lgb.LGBMRegressor(**params)
    regr_multiglb = MultiOutputRegressor(gbm)
    regr_multiglb.fit(train_input_lgb, label_lgb)
    
    model_list.append(
        regr_multiglb
    )

test_input_lgb, test_label_lgb = generate_ts_data_test_for_lgb(df_merged_lgb, term_list)
test_input_lgb.shape

lgb_results_05 = model_list[0].predict(test_input_lgb)
lgb_results_07 = model_list[1].predict(test_input_lgb)
lgb_results_09 = model_list[2].predict(test_input_lgb)

print(round(np.maximum(0.9 * (test_label_lgb - lgb_results_09), (1-0.9)*(lgb_results_09 -test_label_lgb )).mean(), 4)) # 0.0031
print(round(np.mean(test_label_lgb < lgb_results_09), 4), round(0.9 - np.mean(test_label_lgb < lgb_results_09),4))  ### 0.638

print(round(np.maximum(0.7 * (test_label_lgb - lgb_results_07), (1-0.7)*(lgb_results_07 -test_label_lgb )).mean(), 4)) # 0.0051
print(round(np.mean(test_label_lgb < lgb_results_07),4), round(0.7 - np.mean(test_label_lgb < lgb_results_07), 4)) ### 0.640

print(round(np.maximum(0.5 * (test_label_lgb - lgb_results_05), (1-0.5)*(lgb_results_05 -test_label_lgb )).mean(), 4))   # 0.0059
print(round(np.mean(test_label_lgb < lgb_results_05), 4), round(0.5- np.mean(test_label_lgb < lgb_results_05),4)) ### 0.894

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017894 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 257861
[LightGBM] [Info] Number of data points in the train set: 1404, number of used features: 1152
[LightGBM] [Info] Start training from score 0.296533
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 